# Geodetic Tools

This notebook demonstrates the basic workflow for creating an activity rate model from geodetic strain. The methodology
is an implementation of the Seismic Hazard Inferred from Tectonics (SHIFT) (Bird & Liu, 2007; Bird et al., 2010)

### The Geodetic Strain Rate Model

In [ ]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
import numpy as np
from openquake.hmtk.parsers.strain.strain_csv_parser import ReadStrainCsv

strain_file = 'input_data/average_strain_minimal.csv'
fid = open(strain_file, 'r')

for i, row in enumerate(fid.readlines()):
    if i >= 20:
        break
    print row
fid.close()

### Load in the Strain File

In [ ]:
import numpy as np
from openquake.hmtk.parsers.strain.strain_csv_parser import ReadStrainCsv

reader = ReadStrainCsv(strain_file)

strain_data = reader.read_data(scaling_factor=1E-9) # Data reported in nanostrain
print 'Data loaded ok'



In [ ]:
for iloc in range(0, 20):
    print '%8.3f %8.3f %.8e %.8e %.8e %.8e %.8e' % (strain_data.data['longitude'][iloc],
                                                    strain_data.data['latitude'][iloc],
                                                    strain_data.data['exx'][iloc],
                                                    strain_data.data['eyy'][iloc],
                                                    strain_data.data['exy'][iloc],
                                                    strain_data.data['2nd_inv'][iloc],
                                                    strain_data.data['dilatation'][iloc])

### Adding the Tectonic Regionalisaton

To define a seismic moment rate, the volume of each deforming cell must be known. Therefore the seismogenic coupling depth must be defined. This is done using the regionalisation. In the current example the tectonic regionalisation of Kreemer (2003) is used. The cells can be assigned to this regionalisation using as follows:

In [ ]:
from openquake.hmtk.strain.regionalisation.kreemer_regionalisation import KreemerRegionalisation

regionalisation = KreemerRegionalisation()

strain_data = regionalisation.get_regionalisation(strain_data)                              

In [ ]:
for iloc in range(0, 20):
    print '%8.3f %8.3f %.8e %.8e %.8e %.8e %.8e %s' % (strain_data.data['longitude'][iloc],
                                                    strain_data.data['latitude'][iloc],
                                                    strain_data.data['exx'][iloc],
                                                    strain_data.data['eyy'][iloc],
                                                    strain_data.data['exy'][iloc],
                                                    strain_data.data['2nd_inv'][iloc],
                                                    strain_data.data['dilatation'][iloc],
                                                    strain_data.data['region'][iloc])

### Applying the SHIFT Methodology

In [ ]:
from hmtk.strain.shift import Shift

magnitudes = np.arange(5.0, 9.6, 0.1)

modeller = Shift(magnitudes)

Implicit in this methodology are the regionalisation parameters of Bird & Kagan (2004) and Bird et al (2009). The SHIFT methodology assumes that for each cell the activity is modelled via a Tapered Gutenberg-Richter distribution, with the
beta-value and corner magnitude derived according to the region type assigned by Bird & Kagan (2004)

In [ ]:
for key in modeller.regionalisation.keys():
    print key
    print modeller.regionalisation[key]

In [ ]:
# Get the activity rate
modeller.calculate_activity_rate(strain_data, cumulative=False)

In [ ]:
import matplotlib.pyplot as plt

# Show a sample of 8 random cells
sampler = np.random.randint(0, strain_data.get_number_observations(), 5)

plt.figure(figsize=(8.0, 6.0))
for sample in sampler:
    plt.semilogy(magnitudes, modeller.strain.seismicity_rate[sample, :])
plt.xlim(5.0, 9.4)
plt.xlabel('Magnitude', fontsize=12)
plt.ylabel('Incremental Rate', fontsize=12)


### Export the Activity Model to a CSV

In [ ]:
from openquake.hmtk.parsers.strain.strain_csv_parser import WriteStrainCsv

output_file = 'output_data/SHIFT_Demo_Output1.csv'

writer = WriteStrainCsv(output_file)

writer.write_file(modeller.strain, scaling_factor=1E-9)